<h3><b>Capstone Project - Cafes in chennai</b></h3>

In [1]:
import pandas as pd
import folium

In [2]:
import requests

request_parameters = {
    "client_id": 'your foursquare id',
    "client_secret": 'your foursquare client secret',
    "v": '20180605',
    "section": "coffee",
    "near": "Chennai",
    "radius": 5000,
    "limit": 100}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [3]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [4]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Chennai', 'Chennai')

In [5]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 13.102305359732934, 'lng': 80.27718519140059},
  'sw': {'lat': 13.040837610012662, 'lng': 80.23112696288392}},
 35)

In [6]:
d["geocode"]

{'what': '',
 'where': 'chennai',
 'center': {'lat': 13.08784, 'lng': 80.27847},
 'displayString': 'Chennai, Tamil Nādu, India',
 'cc': 'IN',
 'geometry': {'bounds': {'ne': {'lat': 13.150309948059606,
    'lng': 80.30524310784821},
   'sw': {'lat': 12.967193988852129, 'lng': 80.18533922925067}}},
 'slug': 'chennai',
 'longId': '72057594039192463'}

In groups we again have several different items, where type and name are irrelevant for given task. The full path to items is data.json()["response"]["groups"][0]["items"].

Let's print couple of first items to get again better idea of the structure of the data:

In [7]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [8]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [9]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 35


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5343a343498e18de1aeb63ae',
  'name': 'Krispy Kreme Doughnuts',
  'location': {'address': 'Ground Floor, Express Avenue Mall',
   'crossStreet': 'Whites Road',
   'lat': 13.059019182618115,
   'lng': 80.26405141505771,
   'labeledLatLngs': [{'label': 'display',
     'lat': 13.059019182618115,
     'lng': 80.26405141505771}],
   'cc': 'IN',
   'city': 'Chennai',
   'state': 'Tamil Nadu',
   'country': 'India',
   'formattedAddress': ['Ground Floor, Express Avenue Mall (Whites Road)',
    'Chennai',
    'Tamil Nadu',
    'India']},
  'categories': [{'id': '4bf58dd8d48988d148941735',
    'name': 'Donut Shop',
    'pluralName': 'Donut Shops',
    'shortName': 'Donuts',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/donuts_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},

In [10]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4c6930751a6620a1efd2628c',
  'name': 'Cosmopolitan Club',
  'location': {'lat': 13.06503850189714,
   'lng': 80.26936985598189,
   'labeledLatLngs': [{'label': 'display',
     'lat': 13.06503850189714,
     'lng': 80.26936985598189}],
   'cc': 'IN',
   'city': 'Chennai',
   'state': 'Tamil Nadu',
   'country': 'India',
   'formattedAddress': ['Chennai', 'Tamil Nadu', 'India']},
  'categories': [{'id': '4bf58dd8d48988d1f1931735',
    'name': 'General Entertainment',
    'pluralName': 'General Entertainment',
    'shortName': 'Entertainment',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-4c6930751a6620a1efd2628c-1'}

Based on this, a simple for loop over the items is collecting some relevant data from the array, and after that DataFrame is created based on this parsed data.

In [11]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    if not "address" in location:
        continue
    address = location["address"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

found 10 cafes


,uid,name,shortname,address,postalcode,lat,lng
0,5468b5c0498e0e047a092e94,Nicky's Cafe & Fine Pastries,Café,"Fagun Mansion, #74, Ethiraj Salai,",600008,13.063923,80.259764
1,4d57f2a5de8f6dcb5b790591,Coffee Day Lounge,Café,"4, 3rd Avenue, Besant Nagar",600006,13.060967,80.251423
2,516fe170e4b0c56ebcb78cda,Lloyds Tea House - lloyds road,Tea Room,"179, lloyds roads",600086,13.050559,80.255895
3,4f37ad22e4b01139d5fd20e2,Radisson Blu Hotel,Hotel,"Commander In Chief Road, Egmore",600008,13.066939,80.255549
4,4f5278c6e4b0b589381414ed,Anise,Indian,Taj Coromandel,600034,13.058417,80.247327


In [12]:
chennai_center = d["geocode"]["center"]
chennai_center


{'lat': 13.08784, 'lng': 80.27847}

In [15]:
from folium import plugins

# create map of chennai using latitude and longitude values
map_chennai = folium.Map(location=[chennai_center["lat"], chennai_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_chennai)

add_markers(df)
hm_data = df[["lat", "lng"]].values.tolist()
map_chennai.add_child(plugins.HeatMap(hm_data))

map_chennai

In [20]:
lat = 13.08784
lng = 80.27847
map_chennai = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Cafe!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_chennai)
map_chennai